In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.optim as optim
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim = 256, z_dim = 20):
        super(VAE, self).__init__()
        self.img2hid = nn.Linear(input_dim, hidden_dim)
        self.hid2mu = nn.Linear(hidden_dim, z_dim)
        self.hid2std = nn.Linear(hidden_dim, z_dim)
        
        self.z2hid = nn.Linear(z_dim, hidden_dim)
        self.hid2img = nn.Linear(hidden_dim, input_dim)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def Encode(self, x):
        enc_hidden = self.relu(self.img2hid(x))
        mu, std = self.hid2mu(enc_hidden), self.hid2std(enc_hidden)
        return mu, std
        
    def Decode(self, z):
        dec_hidden = self.relu(self.z2hid(z))
        return self.sigmoid(self.hid2img(dec_hidden))

    def forward(self, x):
        mu, std = self.Encode(x)
        epsilon = torch.rand_like(mu)
        reparametrized_z = mu + std * epsilon  #z is actual latent space representation
        output = self.Decode(reparametrized_z)
        return output, mu, std
        

In [29]:
x = torch.randn(4, 28 * 28)
vae = VAE(input_dim = 784, hidden_dim = 256, z_dim = 20)
out, _, _ = vae(x)
out.shape

torch.Size([4, 784])

In [30]:
input_dim = 784
h_dim = 256
z_dim = 20
batch_size = 32
lr = 3e-4
epochs = 10

In [37]:
dataset = datasets.MNIST(root = '/dataset', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True)
model = VAE(input_dim = input_dim, hidden_dim = h_dim, z_dim = z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr = lr)
loss_fn = nn.BCELoss(reduction = 'sum')


In [38]:
for epoch in range(epochs):
    loop =  tqdm(enumerate(train_loader))
    print(epoch)
    for batch_idx, (x, _) in loop:
        x = x.to(device).view(x.shape[0], input_dim)
        x_reconstructed, mu, std = model(x)
        x_reconstruction_loss = loss_fn(x_reconstructed, x)
        kl_div_loss = -torch.sum(1 + torch.log(std.pow(2)) - mu.pow(2) - std.pow(2))
        loss = x_reconstruction_loss + kl_div_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss = loss.item())
        
        if batch_idx % 50 == 0:
            print(loss)
        

5it [00:00, 49.23it/s, loss=1.92e+4]

0
tensor(21014.7988, device='cuda:0', grad_fn=<AddBackward0>)


57it [00:00, 69.19it/s, loss=8.38e+3]

tensor(9809.7783, device='cuda:0', grad_fn=<AddBackward0>)


109it [00:01, 79.95it/s, loss=7.7e+3] 

...

1816it [00:15, 115.84it/s, loss=3.19e+3]

tensor(3052.0754, device='cuda:0', grad_fn=<AddBackward0>)


1864it [00:16, 115.90it/s, loss=3.08e+3]

tensor(3293.8960, device='cuda:0', grad_fn=<AddBackward0>)


1875it [00:16, 114.33it/s, loss=3.35e+3]
